# Muon ML Model using Google Tensorflow

The first step is to load all the required libraries:

In [1]:
import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

Then we load the csv in a pandas dataframe:

In [2]:
muon_dataframe = pd.read_csv('../MuonPOGAnalysisTemplate/output/bxcut_org.csv')

Set feature columns:

In [3]:
def preprocess_features(muon_dataframe):
  """Prepares input features from Muon data set.

  Args:
    muon_dataframe: A Pandas DataFrame expected to contain data
      from muon simulations
  Returns:
    A DataFrame that contains the features to be used for the model.
  """
  selected_features = muon_dataframe[
['Event',
 'n_Primitive',
 '1dtPrimitive.id_r',
 '2dtPrimitive.id_r',
 '3dtPrimitive.id_r',
 '4dtPrimitive.id_r',
 '1dtPrimitive.id_eta',
 '2dtPrimitive.id_eta',
 '3dtPrimitive.id_eta',
 '4dtPrimitive.id_eta',
 '1dtPrimitive.id_phi',
 '2dtPrimitive.id_phi',
 '3dtPrimitive.id_phi',
 '4dtPrimitive.id_phi',
 '1dtPrimitive.phiB',
 '2dtPrimitive.phiB',
 '3dtPrimitive.phiB',
 '4dtPrimitive.phiB',
 '1dtPrimitive.quality',
 '2dtPrimitive.quality',
 '3dtPrimitive.quality',
 '4dtPrimitive.quality',
 'delta_phi12',
 'delta_phi13',
 'delta_phi14',
 'delta_phi23',
 'delta_phi24',
 'delta_phi34']]
  processed_features = selected_features.copy()
  return processed_features.astype(np.float32)


Set target columns:

In [4]:
def preprocess_targets(muon_dataframe):
  """
  Prepares target features (i.e., labels) from muon data set.

  Args:
    muon_dataframe: A Pandas DataFrame expected to contain data
      from the Muon data set.
  Returns:
    A DataFrame that contains the target feature.
  """
  output_targets = pd.DataFrame()
  output_targets["genParticle.pt"] = muon_dataframe["genParticle.pt"]
  return output_targets.astype(np.float32)

For the training set, we'll use the 60% of the dataset

In [5]:
training_examples = preprocess_features(muon_dataframe.head(1184))
training_examples.describe()

,Event,n_Primitive,1dtPrimitive.id_r,2dtPrimitive.id_r,3dtPrimitive.id_r,4dtPrimitive.id_r,1dtPrimitive.id_eta,2dtPrimitive.id_eta,3dtPrimitive.id_eta,4dtPrimitive.id_eta,...,1dtPrimitive.quality,2dtPrimitive.quality,3dtPrimitive.quality,4dtPrimitive.quality,delta_phi12,delta_phi13,delta_phi14,delta_phi23,delta_phi24,delta_phi34
count,1184.0,1184.0,1184.0,1184.0,1184.0,1184.0,1184.0,1184.0,1184.0,1184.0,...,1184.0,1184.0,1184.0,1184.0,1184.0,1184.0,1184.0,1184.0,1184.0,1184.0
mean,599.9,3.0,1.0,1.0,0.8,0.3,-0.0,-0.0,-0.0,-0.0,...,5.1,4.8,3.8,1.7,0.2,0.8,2.2,0.7,1.9,1.3
std,346.0,0.8,0.0,0.2,0.4,0.5,1.1,1.2,1.2,0.8,...,1.4,1.7,2.5,2.5,1.0,1.7,2.1,1.6,2.1,1.9
min,1.0,1.0,1.0,0.0,0.0,0.0,-2.0,-2.0,-2.0,-2.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,301.8,3.0,1.0,1.0,1.0,0.0,-1.0,-1.0,-1.0,0.0,...,5.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,600.5,3.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,6.0,6.0,5.0,0.0,0.0,0.0,1.7,0.0,1.2,0.0
75%,899.2,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,6.0,6.0,6.0,5.0,0.0,0.2,4.1,0.0,3.8,2.7
max,1199.0,4.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,...,6.0,6.0,6.0,6.0,6.3,6.3,6.3,6.3,6.3,6.3


In [6]:
training_targets = preprocess_targets(muon_dataframe.head(1184))
training_targets.describe()

,genParticle.pt
count,1184.0
mean,101.5
std,56.7
min,3.2
25%,53.2
50%,102.4
75%,150.8
max,199.9


And for validation, the remaining data:

In [7]:
validation_examples = preprocess_features(muon_dataframe.tail(790))
validation_examples.describe()

,Event,n_Primitive,1dtPrimitive.id_r,2dtPrimitive.id_r,3dtPrimitive.id_r,4dtPrimitive.id_r,1dtPrimitive.id_eta,2dtPrimitive.id_eta,3dtPrimitive.id_eta,4dtPrimitive.id_eta,...,1dtPrimitive.quality,2dtPrimitive.quality,3dtPrimitive.quality,4dtPrimitive.quality,delta_phi12,delta_phi13,delta_phi14,delta_phi23,delta_phi24,delta_phi34
count,790.0,790.0,790.0,790.0,790.0,790.0,790.0,790.0,790.0,790.0,...,790.0,790.0,790.0,790.0,790.0,790.0,790.0,790.0,790.0,790.0
mean,1600.0,3.1,1.0,1.0,0.8,0.4,-0.0,-0.0,-0.0,0.0,...,5.1,4.8,3.9,1.8,0.2,0.8,2.0,0.6,1.8,1.2
std,231.4,0.8,0.0,0.2,0.4,0.5,1.0,1.2,1.2,0.8,...,1.4,1.7,2.5,2.5,1.0,1.6,2.0,1.4,2.0,1.8
min,1200.0,1.0,1.0,0.0,0.0,0.0,-2.0,-2.0,-2.0,-2.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,1400.2,3.0,1.0,1.0,1.0,0.0,-1.0,-1.0,-1.0,0.0,...,5.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,1600.5,3.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,6.0,6.0,5.0,0.0,0.0,0.0,1.4,0.0,0.9,0.0
75%,1799.8,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,6.0,6.0,6.0,5.0,0.0,0.1,3.8,0.0,3.5,2.3
max,2000.0,4.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,...,6.0,6.0,6.0,6.0,6.3,6.3,6.3,6.3,6.3,6.3


In [8]:
validation_targets = preprocess_targets(muon_dataframe.tail(790))
validation_targets.describe()

,genParticle.pt
count,790.0
mean,100.1
std,57.4
min,3.8
25%,50.3
50%,99.6
75%,148.2
max,199.5


Create the function for loading the data in a TensorFlow model.

In [9]:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    """Trains a linear regression model of multiple features.
  
    Args:
      features: pandas DataFrame of features
      targets: pandas DataFrame of targets
      batch_size: Size of batches to be passed to the model
      shuffle: True or False. Whether to shuffle the data.
      num_epochs: Number of epochs for which data should be repeated. None = repeat indefinitely
    Returns:
      Tuple of (features, labels) for next data batch
    """
    
    # Convert pandas data into a dict of np arrays.
    features = {key:np.array(value) for key,value in dict(features).items()}                                           
 
    # Construct a dataset, and configure batching/repeating.
    ds = Dataset.from_tensor_slices((features,targets)) # warning: 2GB limit
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    # Shuffle the data, if specified.
    if shuffle:
      ds = ds.shuffle(10000)
    
    # Return the next batch of data.
    features, labels = ds.make_one_shot_iterator().get_next()
    for name in features:
        tensor = features[name]
        tensor = tf.expand_dims(tensor,axis=1)
        features[name] = tensor
    return features, labels
    return features, labels

And now we need to create a function to configure feature columns. Right now all columns are numerical, so no worries.

In [10]:
def construct_feature_columns(input_features):
  """Construct the TensorFlow Feature Columns.

  Args:
    input_features: The names of the numerical input features to use.
  Returns:
    A set of feature columns
  """ 
  return set([tf.feature_column.numeric_column(my_feature)
              for my_feature in input_features])

In [11]:
from sklearn.model_selection import ShuffleSplit

In [12]:
split = ShuffleSplit(n_splits=8, test_size=0.3, random_state=42)
for train_index, test_index in split.split(muon_dataframe):
    strat_train_set = muon_dataframe.loc[train_index]
    strat_test_set = muon_dataframe.loc[test_index]

In [13]:
training_examples = preprocess_features(strat_train_set)
training_targets = preprocess_targets(strat_train_set)
validation_examples = preprocess_features(strat_test_set)
validation_targets = preprocess_targets(strat_test_set)

## Try with Random Forest

In [14]:
from tensorflow.contrib.tensor_forest.python import tensor_forest
from tensorflow.python.ops import resources

In [15]:
def train_forest_regression_model(
    num_trees,
    max_nodes,
    steps,
    batch_size,
    training_examples,
    training_targets,
    validation_examples,
    validation_targets):
  """Trains a Random Forest regression model.
  
  In addition to training, this function also prints training progress information,
  as well as a plot of the training and validation loss over time.
  
  Args:
    learning_rate: A `float`, the learning rate.
    steps: A non-zero `int`, the total number of training steps. A training step
      consists of a forward and backward pass using a single batch.
    batch_size: A non-zero `int`, the batch size.
    hidden_units: A `list` of int values, specifying the number of neurons in each layer.
    training_examples: A `DataFrame` containing one or more columns from
      `muon_dataframe` to use as input features for training.
    training_targets: A `DataFrame` containing exactly one column from
      `muon_dataframe` to use as target for training.
    validation_examples: A `DataFrame` containing one or more columns from
      `muon_dataframe` to use as input features for validation.
    validation_targets: A `DataFrame` containing exactly one column from
      `muon_dataframe` to use as target for validation.
      
  Returns:
    A `DNNRegressor` object trained on the training data.
  """

  periods = 20
  steps_per_period = steps / periods
  
  # Create a DNNRegressor object.
  params = tf.contrib.tensor_forest.python.tensor_forest.ForestHParams(num_classes=1, num_features = len(training_examples.columns), regression=True, num_trees=num_trees,max_nodes=max_nodes)
  regressor = tf.contrib.tensor_forest.client.random_forest.TensorForestEstimator(params.fill())
  # Create input functions.
  training_input_fn = lambda: my_input_fn(training_examples, 
                                          training_targets["genParticle.pt"], 
                                          batch_size=batch_size)
  predict_training_input_fn = lambda: my_input_fn(training_examples, 
                                                  training_targets["genParticle.pt"], 
                                                  num_epochs=1,
                                                  shuffle=False)
  predict_validation_input_fn = lambda: my_input_fn(validation_examples, 
                                                    validation_targets["genParticle.pt"], 
                                                    num_epochs=1,
                                                    shuffle=False)

  # Train the model, but do so inside a loop so that we can periodically assess
  # loss metrics.
  print "Training model..."
  print "RMSE (on training data):"

  training_rmse = []
  validation_rmse = []
  for period in range (0, periods):
    # Train the model, starting from the prior state.
    regressor.fit(
        input_fn=training_input_fn,
        steps=steps_per_period)
    # Take a break and compute predictions.
    training_predictions = regressor.predict(input_fn=predict_training_input_fn)
    for item in training_predictions:
        print item
    # ERROR CAUSED BY regressor.predict THAT DOES NOT RETURN A PREDICTION ARRAY BUT A PREDICTION_VARIANCE THAT 
    # HAS NOT THE SAME DIMENSION (THERE IS NOT A CORRELATION 1:1 PREDICTION-TARGET).
    training_predictions = np.array([item['prediction_variance'][0] for item in training_predictions])
    validation_predictions = regressor.predict(input_fn=predict_validation_input_fn)
    validation_predictions = np.array([item['prediction_variance'][0] for item in validation_predictions])
    
    # Compute training and validation loss.
    training_root_mean_squared_error = math.sqrt(
        metrics.mean_squared_error(training_predictions, training_targets))
    validation_root_mean_squared_error = math.sqrt(
        metrics.mean_squared_error(validation_predictions, validation_targets))
    # Occasionally print the current loss.
    print "  period %02d : %0.2f" % (period, training_root_mean_squared_error)
    # Add the loss metrics from this period to our list.
    training_rmse.append(training_root_mean_squared_error)
    validation_rmse.append(validation_root_mean_squared_error)
    if(period==(periods-1)):
      # Output a graph for prediction-target comparison
      fig, ax = plt.subplots()
      ax.scatter(validation_targets,validation_predictions,edgecolors=(0,0,0))
      ax.plot([validation_targets.min(), validation_targets.max()], [validation_targets.min(), validation_targets.max()], 'k--', lw=4)
      ax.set_xlabel('Measured P_t(GeV)')
      ax.set_ylabel('Predicted P_t(GeV)')
      ax.set_title("Random Forest Muon Trigger")
      plt.show()

  print "Model training finished."

  print "Final RMSE (on training data):   %0.2f" % training_root_mean_squared_error
  print "Final RMSE (on validation data): %0.2f" % validation_root_mean_squared_error

  return regressor, training_rmse, validation_rmse

In [16]:
_, training_losses, validation_losses = train_forest_regression_model(
    num_trees=50,
    max_nodes=1000,
    steps=4000,
    batch_size=250,
    training_examples=training_examples,
    training_targets=training_targets,
    validation_examples=validation_examples,
    validation_targets=validation_targets)

Training model...
RMSE (on training data):
{'prediction_variance': array([199.86475], dtype=float32), 'scores': 75.79975}


ValueError: Found input variables with inconsistent numbers of samples: [0, 1381]